# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
#데이터 셋을 데이터 프레임화

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
# 알코올도수, 당도, pH 열을 data에 저장
target = wine['class'].to_numpy()
# class 열을 target에 저장

In [ ]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)
#훈련 세트와 테스트 세트를 8:2로 나눔(test_size=0.2)

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)
#train_input과 train_target을 train_test_split() 함수에 넣어서 검증 세트 생성

In [ ]:
print(sub_input.shape, val_input.shape)
#훈련 세트와 검증 세트의 크기 확인

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
#DecisionTreeClassifier 클래스 import

dt = DecisionTreeClassifier(random_state=42)
#DecisionTreeClassifier 객체 dt 생성
dt.fit(sub_input, sub_target)
#dt 훈련

print(dt.score(sub_input, sub_target))
#훈련 세트 점수 출력
print(dt.score(val_input, val_target))
#검증 세트 점수 출력
#훈련 세트에 과대 적합되어 있으므로 매개변수 바꿔보자!

0.9971133028626413
0.864423076923077


## 교차 검증

- 훈련 세트를 k부분으로 나눠서 교차 검증 수행 -> k-폴드 교차 검증
- 보통 5, 10 - 폴드 교차 검증을 많이 사용함
- 데이터의 80~90%까지 훈련에 사용 가능함
- 사이킷런의 cross_validate() 교차 검증 함수 사용

In [ ]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
#cross_validate()에 인수로 평가 모델 dt, train_input(x값), train_target(y값) 전달
print(scores)
#scores는 return 값으로 fit_time, score_time, test_time 키를 가진 딕셔너리 저장
#fit_time ; 훈련하는 시간
#score_time : 검증하는 시간
#test_score : 검증 폴드의 점수
#교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있음

{'fit_time': array([0.00931716, 0.00749564, 0.00773239, 0.00731683, 0.00710797]), 'score_time': array([0.00109315, 0.00111032, 0.00101209, 0.00106931, 0.00115085]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))
#교차 검증의 최종 점수 출력

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
#cross_validatie() 함수는 분류 모델이므로 StratifiedKFold 사용해서 0과 1 비율 같게 설정
#회귀 모델의 경우 cross_validation 파라미터가 default로 KFold가 지정되어 있는데, 분류 모델은 그러하지 않아서
#cross_validate()는 훈련 세트를 섞어 폴드를 나누지 X
#train_test_split() 함수로 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요 X
#하지만, 교차 검증을 시행한다면 분할기를 지정해야 함 -> StratifiedKFold

print(np.mean(scores['test_score']))
#교차 검증의 최종 점수 출력

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#훈련 세트를 섞은 후 10-폴드 교차 검증 수행(n_splits 기본값은 5이므로 5-폴드 교차 검증)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))
#교차 검증의 최종 점수 출력

0.8574181117533719


## 하이퍼파라미터 튜닝

- 하이퍼파라미터는 사용자 지정 파라미터
- 사이킷런의 그리드 서치를 이용하여 매개변수의 최적값을 조정
- GridSearchCV 클래스는 하이퍼파라미터 탐색과 교차검증을 한 번에 수행

In [ ]:
from sklearn.model_selection import GridSearchCV
#GridSearchCV() 클래스 import

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
#min_impurity_decrease 파라미터의 최적값을 알기 위해 params에 리스트를 value로 하는 딕셔너리 생성
#주어진 값보다 크거나 같은 Gini index의 감소를 다음 split에서 유도가 되면 split
#중간에 stop할 수 있도록하여 decision tree가 overfitting되는 것을 막는다

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
#GridSearchCV의 객체 gs 생성하며, 탐색 대상 모델과 params 변수(실험하고자 하는 파라미터 조합)를 인수로 전달
#n_jobs=-1로 시스템에 있는 모든 CPU 코어를 사용할 수 있도록 지정
#조합을 각각 시험하는 것이므로 조합이 많을수록 훈련속도 느려짐

#사이킷런은 가지고 있는 기본값으로 훈련하지만, 이것으로는 모델 최적화가 어려워
#검증세트 점수로 중간중간 확인했으나, 이것 역시 한계O
#데이터 개수의 확보가 어렵기도 하여 cv(cross validation)으로 교차 검증하면서 하이퍼파라미터를 탐색함
#두 가지 일을 한 번에 하는 것이 바로 GridSearch -> GridSearchCV 클래스

In [ ]:
gs.fit(train_input, train_target)
#gs 객체에 fit() 메서드를 호출하여 min_impurity_decrease 값을 바꿔가며 5번 실행
#GridSearchCV의 cv 매개변수 기본값은 5, min_impurity_decrease 값(5개)마다 5폴드 교차 검증을 수행하여 총 25번 수행

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_
#25개의 모델 중에서 검증 점수가 가장 높은 모델이 best_estimator_ 속성에 저장
print(dt.score(train_input, train_target))
#dt 점수 출력

0.9615162593804117


In [ ]:
print(gs.best_params_)
#그리드 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score'])
#각 매개변수에서 수행한 교차 검증의 평균 점수
#점수는 cv_results_속성의 'mean_test_score' 키에 저장
#5번의 교차 검증으로 얻은 점수 출력(5-폴드 교차 검증)

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
#argmax() 함수를 통해 가장 큰 값의 인덱스를 추출하여 best_index에 저장
print(gs.cv_results_['params'][best_index])
#best_params_ 값과 일치하는지 확인

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          #노드를 분할하기 위한 불순도 감소 최소량 지정
          #이 값보다 크거나 같은 값만큼 감소하지 않는다면 더이상 나무 전개X
          #range(start, stop, step) -> 9개의 배열(stop값 포함X)
          'max_depth': range(5, 20, 1),
          #트리의 깊이 제한
          'min_samples_split': range(2, 100, 10)
          #노드를 나누기 위한 최소 샘플 수
          #샘플이 2밖에 속하지 않는다면 더이상 tree 전개X
          }
#조금 더 복잡한 매개변수 조합 탐색
#min_impurity_decrease, max_depth, min_samples_split에 대해서 각각 9, 15, 10가지 조합
#기본값인 5-폴드 교차 검증 수행하여 9*15*10*5=6750개의 모델 조합 시행

1. 먼저 탐색할 매개변수 지정 (params)
2. 훈련 세트에서 그리드 서치 수행, 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾음 -> 이 조합을 그리드 서치 객체에 저장(ex: best_params_)
3. 그리드 서치는 최상의 매개변수에서 전체 훈련 세트를 사용해 최종 모델 훈련 -> 이 모델도 객체에 저장(ex : best_estimator_)

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
#그리드 서치 실행

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)
#최상의 매개변수 조합 출력

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))
#최상의 교차 검증 점수 출력

0.8683865773302731


### 랜덤 서치

- 매개변수의 값이 수치일 때, 값의 범위나 간격을 미리 정하기 어려울 수 있음
- 혹은, 너무 많은 매개 변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있음
- 랜덤 서치!!
- 매개변수 값의 목록 전달X / 매개변수를 샘플링할 수 있는 확률 분포 객체 전달

In [ ]:
from scipy.stats import uniform, randint
#uniform과 randint 클래스 import

In [ ]:
rgen = randint(0, 10)
#0~10 사이의 범위를 갖는 randint 객체 생성
rgen.rvs(10)
#10개의 숫자(정수) 샘플링

array([4, 7, 6, 8, 9, 3, 8, 3, 1, 4])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)
#1000개의 숫자를 샘플링해서 각 숫자의 개수 출력

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([116, 105,  95, 100,  84,  90,  97,  95, 107, 111]))

In [ ]:
ugen = uniform(0, 1)
#0~1 사이의 범위를 갖는 uniform 객체 생성
ugen.rvs(10)
#10개의 숫자(실수) 샘플링

array([0.07156624, 0.51330724, 0.78244744, 0.14237963, 0.05055468,
       0.13124955, 0.15801332, 0.99110938, 0.08459786, 0.92447632])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
#탐색할 매개변수 딕셔너리 params 생성
#min_samples_leaf 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수
#어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음
#min_impurity_decrease, max_depth, min_samples_split, min_samples_leaf => decision tree의 stop 조건

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#랜덤서치 수행

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
#매개변수 n_iter를 통해 100번 샘플링하여 교차 검증을 수행하여 최적의 매개변수 조합을 찾음

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce351cc0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cccce2f4610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce352da0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce353bb0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
#최적의 매개변수 조합 출력

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))
#최고의 교차 검증 점수 출력

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
#최적의 모델을 dt에 저장
print(dt.score(test_input, test_target))
#dt의 성능 점수 출력
#테스트 세트 점수는 검증 세트에 대한 점수보다 조금 낮은 것이 일반적

0.86


## 확인문제

In [ ]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce351cc0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cccce2f4610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce352da0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cccce353bb0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
